In [32]:
import  keras.layers  as  klayers 
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, LSTM, Input, Embedding, GlobalAveragePooling1D, Concatenate, Activation, Lambda, BatchNormalization, Convolution1D, Dropout
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import initializers
from scipy import stats
from keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models import Word2Vec,doc2vec
import nltk
from quadratic_weighted_kappa import QWK
from sklearn.metrics import cohen_kappa_score


from additional_feature_getter import feature_getter

In [33]:
class Neural_Tensor_layer(Layer):
	def __init__(self,output_dim,input_dim=None, **kwargs):
		self.output_dim=output_dim
		self.input_dim=input_dim
		if self.input_dim:
			kwargs['input_shape']=(self.input_dim,)
		super(Neural_Tensor_layer,self).__init__(**kwargs)

	def build(self,input_shape):
		mean=0.0
		std=1.0
		k=self.output_dim
		d=self.input_dim
		##truncnorm generate continuous random numbers in given range
		W_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(k,d,d))
		V_val=stats.truncnorm.rvs(-2 * std, 2 * std, loc=mean, scale=std, size=(2*d,k))
		self.W=K.variable(W_val)
		self.V=K.variable(V_val)
		self.b=K.zeros((self.input_dim,))
		self.trainable_weights=[self.W,self.V,self.b]

	def call(self,inputs,mask=None):
		e1=inputs[0]
		e2=inputs[1]
		batch_size=K.shape(e1)[0]
		k=self.output_dim
		

		feed_forward=K.dot(K.concatenate([e1,e2]),self.V)

		bilinear_tensor_products = [ K.sum((e2 * K.dot(e1, self.W[0])) + self.b, axis=1) ]

		for i in range(k)[1:]:	
			btp=K.sum((e2*K.dot(e1,self.W[i]))+self.b,axis=1)
			bilinear_tensor_products.append(btp)

		result=K.tanh(K.reshape(K.concatenate(bilinear_tensor_products,axis=0),(batch_size,k))+feed_forward)

		return result

	def compute_output_shape(self, input_shape):
		batch_size=input_shape[0][0]
		return(batch_size,self.output_dim)

In [34]:
class Temporal_Mean_Pooling(Layer): # conversion from (samples,timesteps,features) to (samples,features)
	def __init__(self, **kwargs):
		super(Temporal_Mean_Pooling,self).__init__(**kwargs)
		# masked values in x (number_of_samples,time)
		self.supports_masking=True
		# Specifies number of dimensions to each layer
		self.input_spec=InputSpec(ndim=3)

	def call(self,x,mask=None):
		if mask is None:
			mask=K.mean(K.ones_like(x),axis=-1)

		mask=K.cast(mask,K.floatx())
				#dimension size single vec/number of samples
		return K.sum(x,axis=-2)/K.sum(mask,axis=-1,keepdims=True)

	def compute_mask(self,input,mask):
		return None
	def compute_output_shape(self,input_shape):
		return (input_shape[0],input_shape[2])

In [35]:
fp1=open("glove/glove.6B.300d.txt","r")
glove_emb={}
for line in fp1:
	temp=line.split(" ")
	glove_emb[temp[0]]=np.asarray([float(i) for i in temp[1:]])

print("Embedding done")

Embedding done


In [36]:
EMBEDDING_DIM=300
MAX_NB_WORDS=4000
MAX_SEQUENCE_LENGTH=500
VALIDATION_SPLIT=0.20
DELTA=20


texts=[]
labels=[]
sentences=[]

additional_features=[]

originals = []

In [37]:
essay_type = '4'

fp=open("data/training_set_rel3.tsv",'r', encoding="ascii", errors="ignore")
fp.readline()
for line in fp:
    temp=line.split("\t")
    if(temp[1]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        originals.append(float(temp[6]))
fp.close()

print("range min - ", min(originals) , " ; range max - ", max(originals))

range_min = min(originals)
range_max = max(originals)

fp=open("data/training_set_rel3.tsv",'r', encoding="ascii", errors="ignore")
fp.readline()
sentences=[]
doctovec=[]
for line in fp:
    temp=line.split("\t")
    if(temp[1]==essay_type): ## why only 4 ?? - evals in prompt specific fashion
        texts.append(temp[2])
        labels.append((float(temp[6])-range_min)/(range_max-range_min)) ## why ??  - normalize to range [0-1]
        line=temp[2].strip()
        sentences.append(nltk.tokenize.word_tokenize(line))

fp.close()

range min -  0.0  ; range max -  3.0


In [38]:
labels

[0.0,
 0.0,
 1.0,
 0.6666666666666666,
 0.6666666666666666,
 0.3333333333333333,
 0.0,
 0.6666666666666666,
 0.3333333333333333,
 0.6666666666666666,
 0.6666666666666666,
 0.0,
 0.3333333333333333,
 0.6666666666666666,
 0.0,
 0.3333333333333333,
 0.6666666666666666,
 0.3333333333333333,
 1.0,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 1.0,
 0.3333333333333333,
 0.6666666666666666,
 0.6666666666666666,
 0.3333333333333333,
 0.6666666666666666,
 1.0,
 0.3333333333333333,
 0.6666666666666666,
 0.3333333333333333,
 1.0,
 0.6666666666666666,
 0.3333333333333333,
 0.6666666666666666,
 0.0,
 0.6666666666666666,
 0.0,
 1.0,
 1.0,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.0,
 0.3333333333333333,
 0.6666666666666666,
 0.3333333333333333,
 1.0,
 0.3333333333333333,
 0.3333333333333333,
 0.6666666666666666,
 0.6666666666666666,
 0.0,
 0.0,
 0.3333333333333333,
 1.0,
 0.0,
 0.6666666666666666,
 0.6666666666666666,
 1.0,
 0.6666666666666666,
 0.3333333333

In [39]:
texts[0]

'The author concludes the story with this because gardens cannot grow with snow on the ground. So she she figurers when all the snow is melted shell try to grow another one. I feel as if she wasnt  happy enough to grow another garden I think she should be more interested in growin it then disappointed.    '

In [40]:
print("text labels appended %s" %len(texts))

labels=np.asarray(labels)

for i in texts:
	additional_features.append(feature_getter(i))

text labels appended 1772


In [41]:
additional_features[0]

[2, 60, 3, 20.0, 4.1, 4, 246]

In [42]:
additional_features=np.asarray(additional_features)
for i in sentences:
	temp1=np.zeros((1, EMBEDDING_DIM))
	for w in i:
		if(w in glove_emb):
			temp1+=glove_emb[w]
	temp1/=len(i)
	doctovec.append(temp1.reshape(300,))

In [43]:
doctovec=np.asarray(doctovec)

In [44]:
np.shape(doctovec)

(1772, 300)

In [45]:
tokenizer=Tokenizer() #num_words=MAX_NB_WORDS) #limits vocabulory size
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts) #returns list of sequences
word_index=tokenizer.word_index #dictionary mapping
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print(additional_features.shape,data.shape)

print('Shape of data tensor:', data.shape)

Found 4953 unique tokens.
(1772, 7) (1772, 500)
Shape of data tensor: (1772, 500)


In [46]:
indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
labels=labels[indices]
doctovec=doctovec[indices]
additional_features=additional_features[indices]
validation_size=int(VALIDATION_SPLIT*data.shape[0])

In [47]:
x_train=data[:-validation_size]
y_train=labels[:-validation_size]
doctovec_train=doctovec[:-validation_size]
x_val=data[-validation_size:]
y_val=labels[-validation_size:]
train_ad=additional_features[:-validation_size]
val_ad=additional_features[-validation_size:]
doctovec_val=doctovec[-validation_size:]

In [48]:
# model=Word2Vec(sentences,min_count=1, sg=1,size=EMBEDDING_DIM)

# model.save("3_embeddings")

embedding_matrix = np.zeros((len(word_index), EMBEDDING_DIM))

In [49]:
for word,i in word_index.items():
	if(i>=len(word_index)):
		continue
	if word in glove_emb:
			embedding_matrix[i]=glove_emb[word]
	# if word in model.wv.vocab.keys():
	# 	embedding_matrix[i]=model.wv[word]

vocab_size=len(word_index)

In [50]:
embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=True,
							trainable=False)
side_embedding_layer=Embedding(vocab_size,EMBEDDING_DIM,weights=[embedding_matrix],
							input_length=MAX_SEQUENCE_LENGTH,
							mask_zero=False,
							trainable=False)

In [51]:
def SKIPFLOW(lstm_dim=50, lr=1e-4, lr_decay=1e-6, k=5, eta=3, delta=50, activation="relu", maxlen=MAX_SEQUENCE_LENGTH, seed=None):
	e = Input(name='essay',shape=(maxlen,))
	trad_feats=Input(shape=(7,))
	dtov=Input(shape=(300,))
	embed = embedding_layer(e)
	side_embed = side_embedding_layer(e)
	lstm_layer=LSTM(lstm_dim,return_sequences=True)
	hidden_states=lstm_layer(embed)
	side_hidden_states=lstm_layer(side_embed)
	htm=Temporal_Mean_Pooling()(hidden_states)
	tensor_layer=Neural_Tensor_layer(output_dim=k,input_dim=lstm_dim)
	pairs = [((eta + i * delta) % maxlen, (eta + i * delta + delta) % maxlen) for i in range(maxlen // delta)]
	hidden_pairs = [ (Lambda(lambda t: t[:, p[0], :])(side_hidden_states), Lambda(lambda t: t[:, p[1], :])(side_hidden_states)) for p in pairs]
	sigmoid = Dense(1, activation="sigmoid", kernel_initializer=initializers.glorot_normal(seed=seed))
	coherence = [sigmoid(tensor_layer([hp[0], hp[1]])) for hp in hidden_pairs]
	co_tm=Concatenate()(coherence[:]+[htm])
	dense = Dense(256, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(co_tm)
	dense = Dense(128, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
	dense = Dense(64, activation=activation,kernel_initializer=initializers.glorot_normal(seed=seed))(dense)
	out = Dense(1, activation="sigmoid")(dense)
	model = Model(inputs=[e,trad_feats,dtov], outputs=[out])
	adam = Adam(lr=lr, decay=lr_decay)
	model.compile(loss="mean_squared_error", optimizer=adam, metrics=["MSE"])
	return model


In [52]:
earlystopping = EarlyStopping(monitor="val_mean_squared_error", patience=5)
sf_1 = SKIPFLOW(lstm_dim=50, lr=2e-4, lr_decay=2e-6, k=4, eta=13, delta=50, activation="relu", seed=None)


In [53]:
epochs = 100
# epochs = 1000
hist = sf_1.fit([x_train,train_ad,doctovec_train], y_train, batch_size=1024, epochs=epochs,
                validation_data=([x_val,val_ad,doctovec_val], y_val), callbacks=[earlystopping])


Train on 1418 samples, validate on 354 samples
Epoch 1/100
1418/1418 [==============================] - 9s 6ms/step - loss: 0.0999 - mean_squared_error: 0.0999 - val_loss: 0.0951 - val_mean_squared_error: 0.0951
Epoch 2/100
1418/1418 [==============================] - 3s 2ms/step - loss: 0.0985 - mean_squared_error: 0.0985 - val_loss: 0.0943 - val_mean_squared_error: 0.0943
Epoch 3/100
1418/1418 [==============================] - 3s 2ms/step - loss: 0.0975 - mean_squared_error: 0.0975 - val_loss: 0.0937 - val_mean_squared_error: 0.0937
Epoch 4/100
1418/1418 [==============================] - 3s 2ms/step - loss: 0.0967 - mean_squared_error: 0.0967 - val_loss: 0.0928 - val_mean_squared_error: 0.0928
Epoch 5/100
1418/1418 [==============================] - 3s 2ms/step - loss: 0.0957 - mean_squared_error: 0.0957 - val_loss: 0.0918 - val_mean_squared_error: 0.0918
Epoch 6/100
1418/1418 [==============================] - 3s 2ms/step - loss: 0.0946 - mean_squared_error: 0.0946 - val_loss: 0.0

In [54]:
y_pred=sf_1.predict([x_val,val_ad,doctovec_val])


In [55]:
y_val_fin = [int(round(a*(range_max-range_min)+range_min)) for a in y_val]

In [57]:
y_pred_fin =[int(round(a*(range_max-range_min)+range_min)) for a in y_pred.reshape(354).tolist()]

In [58]:
print(cohen_kappa_score(y_val_fin,y_pred_fin,weights="quadratic"))

0.7902661772781133


In [60]:
sf_1.save('4_model.h5')

In [29]:
y_pred*(range_max-range_min)+range_min

array([[37.655647],
       [37.61193 ],
       [43.24997 ],
       [40.21272 ],
       [38.533585],
       [36.21241 ],
       [36.59495 ],
       [35.90928 ],
       [41.424652],
       [38.39617 ],
       [30.6541  ],
       [33.754498],
       [29.771978],
       [38.186462],
       [35.046486],
       [31.218304],
       [40.664257],
       [41.2769  ],
       [40.152946],
       [34.431534],
       [37.73353 ],
       [40.108177],
       [40.490265],
       [35.33967 ],
       [36.986275],
       [36.98564 ],
       [35.761414],
       [38.322144],
       [34.203346],
       [45.936516],
       [41.50494 ],
       [34.36251 ],
       [39.254936],
       [42.239418],
       [38.22659 ],
       [37.256927],
       [40.80839 ],
       [29.887756],
       [38.07134 ],
       [37.655693],
       [37.428432],
       [37.049194],
       [34.822044],
       [42.396366],
       [39.12339 ],
       [37.936356],
       [39.9822  ],
       [38.402573],
       [29.93917 ],
       [41.666664],


In [30]:
y_pred_fin

[38,
 38,
 43,
 40,
 39,
 36,
 37,
 36,
 41,
 38,
 31,
 34,
 30,
 38,
 35,
 31,
 41,
 41,
 40,
 34,
 38,
 40,
 40,
 35,
 37,
 37,
 36,
 38,
 34,
 46,
 42,
 34,
 39,
 42,
 38,
 37,
 41,
 30,
 38,
 38,
 37,
 37,
 35,
 42,
 39,
 38,
 40,
 38,
 30,
 42,
 36,
 40,
 37,
 34,
 35,
 42,
 38,
 40,
 37,
 40,
 41,
 36,
 38,
 36,
 31,
 40,
 37,
 35,
 41,
 35,
 37,
 37,
 34,
 34,
 44,
 34,
 40,
 30,
 34,
 36,
 43,
 35,
 45,
 36,
 36,
 27,
 41,
 36,
 41,
 32,
 34,
 28,
 31,
 39,
 37,
 43,
 43,
 37,
 42,
 44,
 35,
 37,
 36,
 42,
 35,
 34,
 40,
 30,
 41,
 40,
 41,
 33,
 31,
 37,
 32,
 33,
 37,
 38,
 31,
 41,
 31,
 34,
 39,
 38,
 34,
 36,
 36,
 42,
 43,
 38,
 38,
 36,
 37,
 41,
 29,
 31,
 36,
 26,
 36,
 34,
 33,
 36,
 34,
 36]

In [31]:
y_val_fin

[40,
 45,
 42,
 45,
 45,
 36,
 40,
 35,
 44,
 40,
 10,
 30,
 30,
 43,
 38,
 20,
 40,
 44,
 36,
 36,
 37,
 40,
 40,
 35,
 40,
 36,
 35,
 35,
 32,
 44,
 40,
 40,
 40,
 43,
 42,
 36,
 40,
 30,
 44,
 40,
 37,
 36,
 33,
 41,
 40,
 40,
 45,
 37,
 32,
 44,
 50,
 34,
 34,
 38,
 42,
 45,
 40,
 39,
 36,
 35,
 34,
 30,
 43,
 42,
 37,
 36,
 31,
 38,
 42,
 36,
 32,
 40,
 40,
 35,
 45,
 40,
 45,
 28,
 37,
 36,
 40,
 35,
 46,
 40,
 34,
 27,
 40,
 37,
 42,
 33,
 34,
 20,
 26,
 38,
 28,
 36,
 42,
 40,
 45,
 50,
 30,
 40,
 34,
 35,
 38,
 37,
 40,
 31,
 36,
 40,
 40,
 40,
 30,
 40,
 40,
 20,
 34,
 40,
 33,
 32,
 44,
 36,
 41,
 37,
 31,
 30,
 37,
 50,
 34,
 40,
 47,
 37,
 36,
 38,
 28,
 25,
 40,
 27,
 45,
 33,
 36,
 32,
 30,
 34]

In [64]:
from keras.utils import plot_model
plot_model(sf_1)


ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [65]:
sf_1.save_weights('4_weights.h5')